## Instalacja Sparka na platformie Colab

In [0]:
#Instalacja Sparka na colabie
!git clone https://github.com/djkormo/colab-examples.git

In [0]:
!bash colab-examples/spark/install.bash

In [0]:
# based on https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_ME'] = '/content/spark-2.4.4-bin-hadoop2.7'

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("First App") \
    .master("local[*]")\
    .getOrCreate()


In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## Wstęp

PySpark jest świetnym językiem do przeprowadzania eksploracyjnych analiz danych na dużą skalę, budowania potoków uczenia maszynowego i tworzenia ETL dla platformy danych. Jeśli znasz już Python i biblioteki takie jak Pandas, to PySpark jest świetnym językiem do nauki w celu stworzenia bardziej skalowalnych analiz i potoków. 
Użyjemy Databricks dla środowiska Spark i zbioru danych NHL z serwisu Kaggle, jako źródła danych do analizy. Tekst opisuje jak odczytywać i zapisywać dane w ramkach danych Spark, tworzyć transformacje i agregacje tych ramek, wizualizować wyniki i wykonywać regresję liniową. Dowiesz się także, jak łączyć regularny kod Pythona z PySpark w skalowalny sposób, używając funkcji Pandas UDF. Aby wszystko było proste, skoncentrujemy się na przetwarzaniu wsadowym i unikniemy niektórych komplikacji związanych z strumieniowym przesyłaniem danych.

### Środowisko
Istnieje kilka różnych opcji i uruchamiania Spark:
- Self Hosted: można samemu ustawić klaster przy użyciu maszyn typu bare metal lub maszyn wirtualnych. Apache Ambari to użyteczny projekt w takim przypadku, natomiast to podejście nie jest zalecane do szybkiego rozpoczęcia pracy z platformą Spark. 
- Dostawcy usług: Większość dostawców usług w chmurze oferuje klastry Spark: AWS ma EMR, a GCP ma DataProc. W tym przypadku można stworzyć interaktywne środowisko szybciej, niż w przypadku self-hosting.
- Rozwiązania dostawców: Firmy, w tym Databricks i Cloudera, zapewniają rozwiązania Spark, dzięki czemu można łatwo rozpocząć pracę ze Spark.

Wybór rozwiązania dla zastosowania różni się w zależności od zakładanego stopnia bezpieczeństwa, kosztów i istniejącej infrastruktury. Jeśli próbujesz zacząć korzystać ze środowiska, w którym chcesz się uczyć, proponuję skorzystać z wersji społecznościowej Databricks.
Dzięki środowisku Databricks CE można łatwo rozpocząć pracę z klastrem Spark i środowiskiem notatników Jupyter. W tym instruktażu wykorzystano klaster z modułem wykonawczym Spark 2.3.1 i Python 3.6. Uwaga: aby uruchomić kod wykorzystywany w samouczku, jest potrzebne środowisko Spark przynajmniej w wersji 2.3, aby można było korzystać z funkcjonalności Pandas UDF.


### Struktura Spark Dataframes
Kluczowym typem danych wykorzystywanym w PySpark jest Spark Dataframe. Obiekt ten może być traktowany, jako tabela rozproszona w klastrze i ma funkcjonalność podobną do typu DataFrame w R lub Pandas. Jeśli chcemy wykonać obliczenia rozproszone za pomocą PySpark, musimy wykonywać operacje na typach Spark DataFrame. 
Możliwe jest także użycie ramek danych (ang. data frames) pakietu Pandas podczas pracy w środowisku Spark, poprzez wywołanie funkcji toPandas() na ramce danych Spark, która zwraca obiekt Pandas DataFrame. Zasadniczo należy jednak unikać zastosowania tej funkcji, z wyjątkiem pracy z małymi ramkami danych, ponieważ pobiera ona cały obiekt do pamięci na tylko jednym węźle.
Jedną z kluczowych różnic między ramkami danych Pandas i Sparka jest różnica pod względem podejścia do sposobu przetwarzania danych. W PySpark operacje są opóźnione, dopóki wynik nie jest faktycznie potrzebny w potoku. Na przykład można określić operacje ładowania zestawu danych z S3 i stosowania pewnej liczby przekształceń do ramki danych, ale operacje te nie zostaną natychmiast zastosowane. Zamiast tego rejestrowany jest graf przekształceń, a gdy dane są faktycznie potrzebne, na przykład podczas zapisywania wyników z powrotem do S3, transformacje są stosowane, jako pojedyncza operacja potoku. To podejście jest stosowane w celu uniknięcia pobierania pełnej ramki danych do pamięci i umożliwia bardziej efektywne przetwarzanie na klastrze. W przypadku ramek danych Pandas wszystko jest pobierane do pamięci i każda operacja Pandas jest natychmiast stosowana.
Ogólnie rzecz biorąc najlepszą praktyką jest unikanie natychmiastowo wykonywanych operacji w Sparku, o ile jest to możliwe, ponieważ ogranicza to efektywność dystrybucji potoku.


## Załadowanie danych

Jednym z pierwszych kroków jest ładowanie zestawu danych do obiektu DataFrame. Po wczytaniu danych do ramki danych można zastosować transformacje, przeprowadzić analizę i modelowanie, tworzyć wizualizacje i zapisywać wyniki. W Pythonie można ładować pliki bezpośrednio z lokalnego systemu plików przy użyciu Pandas.

```
import pandas as pd
pd.read_csv("dataset.csv")
```

W PySpark ładowanie pliku CSV jest nieco bardziej skomplikowane. W środowisku rozproszonym nie ma lokalnego magazynu, a zatem do określenia ścieżki pliku należy użyć rozproszonego systemu plików, takiego jak HDFS, magazynu plików Databricks (DBFS) lub S3.
Generalnie, używając PySpark, wiele projektów pracuje z danymi w gromadzonymi w S3. Wiele baz danych zapewnia funkcje zapisu i odczytu do S3, a także można użyć konsoli AWS do przenoszenia plików z komputera lokalnego do S3. Pierwszym krokiem jest przesłanie pliku CSV, który będzie przetwarzany. Następnym krokiem jest wczytanie pliku CSV do obiektu DataFrame Spark. Ten fragment kodu określa ścieżkę pliku CSV i przekazuje szereg argumentów funkcji odczytu w celu poprawnego przetworzenia pliku. Ostatni krok wyświetla podzbiór załadowanego obiektu DataFrame, jest to podobne działania do funkcji df.head() w Pandas.

In [0]:
file_location = '/content/colab-examples/spark/data/nhl-game-data/game_skater_stats.csv'

In [0]:
# załadowanie danych z pliku lokalnego CSV
df = spark.read.format("CSV"). \
                option('inferSchema', True). \
                option('header', True). \
                load(file_location)

In [0]:
# zastosowanie funkcji display() nie daje czytelnych wyników (w przypadku Databricks funkcja dispaly() działa znacznie lepiej)
display(df.take(5))

In [0]:
df.limit(5).toPandas()

In [0]:
# zastosowanie limitu jest istotne w przypadku dużej ilości encji w obiekcie Spark DataFrame
# funkcja toPandas() kopiuje encje z obiektu Spark DataFrame do obiektu Pandas DataFrame
# należy mieć na uwadze fakt, że w takim przypadku można przepełnić obiekt Pandas DataFrame

df.limit(num=5).toPandas().head()

## Zapis danych

Powszechnie używanym formatem przechowywania danych w przypadku Big Data jest Parquet. Podobnie jest w przypadku Sparka, ponieważ Parquet jest formatem, który przechowuje metadane dotyczące typów danych kolumn, oferuje kompresję plików i zaprojektowanym do współpracy z Sparkiem. 

AVRO to kolejny format, który dobrze pasuje do Sparka. Poniższy fragment pokazuje, jak korzystając z obiektu DataFrame utworzonego w poprzednim fragmencie kodu, zapisać dane jako plik w formacie Parquet, a następnie ponownie załadować dane do obiektu DataFrame z zapisanego pliku.

In [0]:
# zapis lokalny do formatu Parquet
df.write.save('/content/colab-examples/spark/data/nhl-game-parquet/game_stats', format='parquet', mode="overwrite")

Wynik następnego kroku jest taki sam – czyli wczytanie danych z pliku do obiektu DataFrame - ale przepływ wykonania jest znacząco inny. Podczas odczytu plików CSV do ramek danych, Spark wykonuje operację w trybie *eager* (czyli wykonywany natychmiastowo), co oznacza, że wszystkie dane są wczytywane do pamięci przed rozpoczęciem kolejnego etapu, podczas gdy „leniwe” podejście jest używane podczas odczytu plików w formacie Parquet. Ogólnie rzecz biorąc, chcemy unikać operacji typu eager podczas pracy ze Spark, i jeśli istnieje potrzeba przetwarzania dużych plików CSV, najpierw transformujemy zestaw danych do formatu Parquet przed wykonaniem reszty zaprojektowanego potoku.

In [0]:
df = spark.read.load("/content/colab-examples/spark/data/nhl-game-parquet/game_stats")
display(df)

Często istnieje potrzeba przetwarzania dużej ilości plików, np. setek plików typu Parquet umieszczonych w określonej ścieżce lub katalogu określonym systemie plików. Na platformie Spark można umieścić symbol wieloznaczny w ścieżce do przetwarzania zbioru plików. Na przykład:

```
df = spark.read.load("s3a://my_bucket/game_skater_stats/*.parquet")
```

### Lokalizacja zapisywanych danych
Podobnie jak w przypadku odczytu danych za pomocą Sparka, nie zaleca się zapisywania danych w lokalnym systemie plików podczas korzystania z interfejsu PySpark. Zamiast tego należy użyć rozproszonego systemu plików, takiego jak S3 lub HDFS. Jeśli zamierzamy intensywniej przetwarzać wyniki działania za pomocą Sparka, wtedy Parquet jest dobrym formatem do zapisywania danych z obiektu DataFrame. Poniższy fragment pokazuje, jak zapisać dataframe do DBFS i S3 w formacie Parquet.

In [0]:
# AWS S3, zapis do formatu Parquet
# zastosowanie przełącznika mode pozwala na nadpisywanie lub dołączanie danych do istniejących plików
df.write.parquet("s3a://sqlsaturday/game_skater_stats", mode="overwrite")

Podczas zapisywania ramki danych w formacie Parquet, często jest ona podzielona na kilka plików. Jeśli potrzebujesz wyników zapisanych w pliku CSV, wymagana jest nieco inna procedura. Jedną z głównych różnic w tym podejściu jest to, że wszystkie dane zostaną przyciągnięte do pojedynczego węzła, zanim zostaną wyprowadzone do pliku CSV. Takie podejście jest zalecane, gdy zachodzi potrzeba zapisania małej ramki danych i przetworzenia jej poza Sparkiem. Poniższy fragment pokazuje, jak zapisać dataframe jako pojedynczy plik CSV w systemach plików DBFS i S3.

In [0]:
# zapis lokalny do formatu CSV
df.write.save('./data/nhl-game-csv/game_stats.csv', format='csv',  mode="overwrite")

In [0]:
# https://www.oreilly.com/library/view/learning-spark/9781449359034/ch04.html
# AWS S3 zapis do formatu CSV
# Kolejność operacji ma znaczenie: np. mode przed save :) dlaczego?
df.coalesce(1).write.format("com.databricks.spark.csv")\
                    .option("header", "true")\
                    .mode("overwrite")\
                    .save("s3a://sqlsaturday/game_skater_stats.csv")

Innym typowym wyjściem skryptów Spark jest baza danych typu NoSQL, taka jak Cassandra, DynamoDB lub Couchbase. Jedną z metod, jest zapisanie obiektu dataframe do S3, a następnie uruchamianie procesu ładowania, który przekazuje systemowi NoSQL, aby załadował dane z określonej ścieżki w systemie AWS S3.

## Przetwarzanie danych

Na obiektach DataFrame Spark można wykonywać wiele różnych operacji, podobnie jak w przypadku obiektów DataFrame Pandas. Jednym ze sposobów wykonywania operacji na DataFrame Spark jest użycie Spark SQL, który umożliwia uruchomienie zapytania na obiektach DataFrame tak, jakby były one tabelami. Poniższy fragment pokazuje, jak znaleźć najlepszych graczy w załadowanym zestawie danych NHL.

In [0]:
df.createOrReplaceTempView('statystyki')

In [0]:
spark.sql("""
   select player_id, sum(1) as roz_gry, sum(goals) as gole
   from statystyki
   group by 1
   order by 3 desc
   limit 5
""").show()

Rezultatem jest lista identyfikatorów graczy (player_id), liczba występów w grach (roz_gry) oraz liczba goli strzelonych w tych grach (gole). Jeśli chcemy pokazać imiona graczy, musimy załadować dodatkowy plik, udostępnić go, jako widok tymczasowy, a następnie zastosować metodę JOIN przy pomocy Spark SQL.
W powyższym fragmencie użyto polecenia *limit*, aby wyświetlić próbkę zestawu danych, ale możliwe jest również przypisanie wyników do innego DataFrame, który może być użyty w kolejnych etapach potoku. Poniższy kod pokazuje, jak wykonać te kroki. Najpierw wyniki zapytania są przypisywane do nowej ramki danych, która jest następnie przypisywana do widoku tymczasowego i dołączana jest do kolekcji nazw graczy.

### Wybór najlepszego gracza

In [0]:
# nazwiska graczy
plik_gracz_info = '/content/colab-examples/spark/data/nhl-game-data/player_info.csv'
nazwiska = spark.read.format("CSV").option('inferSchema', True).option('header', True).load(plik_gracz_info)
nazwiska.toPandas().head()

In [0]:
# najlepsi gracze
df.createOrReplaceTempView('statystyki')

gracze = spark.sql("""
   select player_id, sum(1) as roz_gry, sum(goals) as gole
   from statystyki
   group by 1
   order by 3 desc
   limit 5
""")

gracze.createOrReplaceTempView('gracze')

nazwiska.createOrReplaceTempView('nazwiska')

ranking = spark.sql("""
    select g.player_id, gole, firstName, lastName
    from gracze g
    join nazwiska n
      on g.player_id = n.player_id
    order by 2 desc
""")

ranking.toPandas().head()

Wynik tego procesu pokazano powyżej, identyfikując Alexa Ovechkina, jako najlepszego gracza w NHL, w oparciu o zestaw danych Kaggle.
Istnieją operacje wykonywane na obiektach DataFrame Spark dla typowych zadań, takich jak dodawanie nowych kolumn, usuwanie kolumn, wykonywanie operacji JOIN i tworzenia agregacji i statystyki analitycznej, ale na początku swojej drogi ze Sparkiem łatwiej jest wykonywać te operacje przy użyciu SparkSQL. Przeniesienie kodu z Python do PySpark jest również łatwe, jeśli korzystasz już z bibliotek takich jak PandaSQL (https://pypi.org/project/pandasql/) lub framequery (https://pypi.org/project/framequery/) do manipulowania ramkami danych Pandas za pomocą SQL.
Podobnie jak większość operacji na DataFrame Spark, w przypadku Spark SQL operacje są wykonywane w trybie leniwego wykonywania (ang. lazy execution), co oznacza, że kroki w języku SQL nie będą wykonywane aż do momentu, gdy potrzebny jest ich wynik. Spark SQL zapewnia świetny sposób na poznawanie PySpark, bez konieczności uprzedniego poznawania nowej biblioteki.
Korzystając z Databricks, można także tworzyć wizualizacje bezpośrednio w notatniku, bez jawnego korzystania z bibliotek wizualizacyjnych. 
Możemy na przykład wykreślić średnią liczbę goli na mecz za pomocą kodu SparkSQL poniżej.

In [0]:
# goli na mecz
gra_gole = spark.sql("""
   select cast(substring(game_id, 1, 4) || '-' || substring(game_id, 5, 2) || '-01' as Date) as month,
          sum(goals)/count(distinct game_id) as gole_na_mecz
   from statystyki
   group by 1
   order by 1
""")

gra_gole.toPandas().head()

In [0]:
# goli na strzał i ilość graczy
strzal_gol = spark.sql("""
   select cast(gole/strzaly * 50 as int)/50.0 as gole_strzal, sum(1) as il_graczy
   from (
     select player_id, sum(shots) as strzaly, sum(goals) as gole
     from statystyki
     group by 1
     having gole >= 5
   )
   group by 1
   order by 2 desc
""")

strzal_gol.toPandas().head(10)

W przypadku notatników w środowisku Databricks, początkowo wyniki operacji wyjściowych są wyświetlane w postaci tabeli, ale można również użyć funkcjonalności wizualizacyjnych, aby przekształcić dane wyjściowe w różne wizualizacje, takie jak np. wykres słupkowy. Takie podejście nie wspiera każdego typu wizualizacji, ale znacznie ułatwia ona eksploracyjną analizę danych w Sparku. Jeśli zajdzie taka potrzeba, można użyć funkcji toPandas(), aby utworzyć ramkę danych Pandas (utworzy się na węźle driver), co oznacza, że dowolna biblioteka wizualizująca dane stosowana w języku Python może być używana do wizualizacji wyników w środowisku Spark. Jednak to podejście powinno być stosowane tylko dla małych zbiorów danych (czyli małych obiektów DataFrame), ponieważ wszystkie dane są ładowane do pamięci w węźle driver (ang. driver node).

## MLlib - Regresja Liniowa

Jednym z typowych przypadków użycia Pythona do zastosowań w Data Science jest budowanie modeli predykcyjnych. O ile zastosowanie pakietu scikit-learn jest bardzo dobre podczas pracy z danymi przechowanymi w Pandas DataFrame, nie skaluje się on do dużych zestawów danych w środowisku rozproszonym. Podczas budowania modeli predykcyjnych z PySpark i dużych zestawów danych, MLlib jest preferowaną biblioteką, ponieważ natywnie działa na obiektach Spark DataFrame. Nie każdy algorytm uczenia maszynowego zawarty w bibliotece scikit-learn jest dostępny w MLlib, ale istnieje wiele różnych opcji obejmujących wiele przypadków użycia.
Aby użyć jednego z algorytmów uczenia maszynowego zawartych w MLib, należy skonfigurować DataFrame za pomocą wektora cech i etykiety w postaci skalara. Po przygotowaniu można użyć funkcji fit(), aby wyszkolić model. W poniższym fragmencie przedstawiono sposób łączenia kilku kolumn obiektu DataFrame w jeden wektor cech za pomocą funkcji VectorAssembler(). Korzystamy z wynikowej ramki danych, aby wywołać funkcję fit(), a następnie wygenerować statystyki podsumowujące działanie modelu.


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [0]:
df.columns

In [0]:
kolumy_cechy = VectorAssembler(inputCols=['shots', 'hits', 'assists', 
                                          'penaltyMinutes', 'timeOnIce', 
                                          'takeaways'], 
                               outputCol='cechy')

In [0]:
trening_df = kolumy_cechy.transform(df)

In [0]:
regresja_liniowa = LinearRegression(featuresCol='cechy', labelCol='goals')

In [0]:
model_rl = regresja_liniowa.fit(trening_df)

In [0]:
trening_podsumowanie = model_rl.summary

In [0]:
print("Współczynniki: " + str(model_rl.coefficients))
print("RMSE: %f" % trening_podsumowanie.rootMeanSquaredError)
print("R2: %f" % trening_podsumowanie.r2)

In [0]:
trening_podsumowanie.residuals.show()

Model prognozuje, ile goli uzyska gracz, na podstawie liczby strzałów, czasu w grze i innych czynników z historii graczy. Jednak wydajność tego modelu jest raczej niska, co wyjaśnia błąd średniej kwadratowej (RMSE) wynoszący 0,375 i wartość $R^2$ wynosząca 0,125. Współczynnikiem o największej wartości była kolumna 'shots', ale to nie wystarczy, aby model był wystarczająco dokładny.

Podczas budowania potoku uczenia maszynowego z PySpark należy uwzględnić dodatkowe kroki, w tym uczenie i testowanie na róznych zbiorach danych, strojenie hiperparametrów i finalnie zapisywanie modeli. Powyższy fragment jest punktem wyjścia do rozpoczęcia pracy z MLlib.

## Pandas UDF

https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html

Jednym z mechanizmów, które umożliwiają wykonywanie obliczeń rozproszonych za pomocą Pandas DataFrame w środowisku Spark jest Pandas UDF (ang. Pandas User Defined Functions). Ogólny sposób działania UDF polega na tym, że najpierw dzieli się Spark DataFrame na partycje, używając funkcji groupBy, a każda partycja jest wysyłana do węzła roboczego (ang. worker) i tłumaczona jest na Pandas DataFrame, który jest następnie przekazywany do UDF. Z kolei UDF zwraca przekształcony obiekt Pandas DataFrame, który jest połączeniem wszystkich utworzonych partycji, a następnie jest przekształcany z powrotem w obiekt Spark DataFrame. Dzięki temu można użyć bibliotek Pythona, które wymagają pakietu Pandas i umożliwić skalowanie do dużych zbiorów danych, o ile potrafi się dobrze spartycjonować obiekt Spark DataFrame. Mechanizm Pandas UDF został wprowadzony w Sparku 2.3.

Dopasowanie krzywej to typowe zadanie, które jest często wykonywane w badaniu danych. Poniższy fragment kodu pokazuje, w jaki sposób dopasować krzywą, aby opisać związek między liczbą uderzeń i trafień, które gracz osiąga w trakcie gry. Kod pokazuje, w jaki sposób można wykonać to zadanie dla pojedynczego gracza, wywołując funkcję toPandas() na zestawie danych przefiltrowanym do pojedynczego gracza. Wynik tego kroku to dwa parametry (współczynniki regresji liniowej), które próbują opisać związek między tymi zmiennymi.

In [0]:
# tworzenie liniowego dopasowania dla pojedynczego użytkownika

jeden_gr = spark.sql("""
  select * from statystyki
  where player_id = 8467412
""").toPandas()

In [0]:
from scipy.optimize import leastsq
import numpy as np

In [0]:
def fit(params, x, y):
    return (y - (params[0] + x * params[1]))

wynik = leastsq(fit, [1, 0], args=(jeden_gr.shots, jeden_gr.hits))

In [0]:
print(wynik)

Jeśli chcemy obliczyć tę krzywą dla każdego gracza, dysponując ogromnym zbiorem danych, wywołanie toPandas() nie powiedzie się z powodu wyjątku spowodowanego brakiem pamięci. Można przeskalować tę operację do całego zestawu danych, wywołując groupby() na kolumnie 'player_id', a następnie stosując kod Pandas UDF jak to pokazano poniżej. Funkcja przyjmuje jako daną wejściową obiekt Pandas Dataframe, który opisuje statystykę rozgrywki pojedynczego gracza, i zwraca obiekt DataFrame zawierający podsumowanie, które zawiera player_id i dopasowane współczynniki. Każdy z obiektów Pandas DataFrame zawierający podsumowanie jest następnie łączony w obiekt Spark DataFrame, który jest wyświetlany na końcu fragmentu kodu. Dodatkowym elementem konfiguracji w przypadku korzystania z Pandas UDF jest definiowanie schematu dla wynikowego obiektu DataFrame, gdzie schemat opisuje format Spark DataFrame, generowanego w ostatnim kroku.

In [0]:
# zastosowanie Pandas UDF
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
import pandas as pd

In [0]:
schemat = StructType([StructField('ID', LongType(), True),
                      StructField('p0', DoubleType(), True),
                      StructField('p1', DoubleType(), True)])

In [0]:
@pandas_udf(schemat, PandasUDFType.GROUPED_MAP)
def analiza_gracza(jeden_gr):
    if (len(jeden_gr.shots) <= 1):
        return pd.DataFrame({'ID': [jeden_gr.player_id[0]], 'p0': [ 0 ], 'p1': [ 0 ]})
    
    wynik = leastsq(fit, [1,0], args=(jeden_gr.shots, jeden_gr.hits))
    return pd.DataFrame({'ID': [jeden_gr.player_id[0]], 'p0': [wynik[0][0]], 'p1': [wynik[0][1]]})

In [0]:
gracz_df = df.groupby('player_id').apply(analiza_gracza)

In [0]:
gracz_df.limit(num=10).toPandas()

Dane wyjściowe tego procesu przedstawiono powyżej. Istnieje teraz obiekt DataFrame, który podsumowuje dopasowanie krzywej dla każdego gracza i można uruchomić tę operację na dużym zbiorze danych. Podczas pracy z dużymi zbiorami danych ważne jest, aby wybrać lub wygenerować klucz partycji, aby uzyskać kompromis między liczbą i wielkością partycji danych.

## Dobre praktyki przy korzystaniu z PySpark

Omówiliśmy niektóre z typowych zadań związanych z korzystaniem z PySpark. Poniżej można się zapoznać z kilkoma poradami ułatwiającymi przejście od analizy danych za pomocą Pythona do analizy danych w PySpark. Oto niektóre z najlepszych praktyk:

- Unikaj słowników, korzystaj z obiektów DataFrame: używanie typów danych języka Python, takich jak słowniki, oznacza, że kod nie może być wykonywany w środowisku rozproszonym. Zamiast używać kluczy (key:value) do indeksowania wartości w słowniku, należy rozważyć dodanie kolejnej kolumny do obiektu DataFrame, która może być używana jako filtr.
- Używaj oszczędnie funkcji toPandas(): wywołanie tej funkcji spowoduje załadowanie wszystkich danych do pamięci w węźle drivera (ang. driver node) i uniemożliwi wykonywanie operacji w trybie rozproszonym. Funkcję tę można używać, gdy dane zostały już zagregowane i użytkownik chce skorzystać z narzędzi Pythona do ich wizualizacji, ale nie powinna być używana do dużych obiektów DataFrame.
- Unikaj pętli, czyli składni for: Jeśli to możliwe, lepiej przeprogramować logikę pętli za pomocą wzorca groupby-apply, aby zrównoleglone wykonanie kodu było możliwe. Zastosowanie tego wzorca w przypadku kodu w jezyku Python przekłada się na czytelność kodu i jego łatwiejsze przeniesienie do środowiska Spark.
- Spróbuj zminimalizować operacje wykonywane natychmiastowo: aby potok był możliwie jak najbardziej skalowalny, dobrze jest unikać operacji określanych jako *eager*, które pobierają pełne ramki danych do pamięci. Czytanie plików CSV jest operacją typu eager, dlatego efektywniej jest zapisywać ramki danych w formacie Parquet, a następnie ponowne załadować dane z pliku Parquet, a to wszystko w celu otrzymania bardziej skalowalnych potoków.
- Użyj mechanizmów framequery/pandasql, aby ułatwić przenoszenie kodu: jeśli pracujesz z kodem Pythona stworzonym przez innych, może być trudno odczytać, co mają na celu niektóre operacje napisane z zastodowaniem Pandas. Jeśli planujesz przeniesienie kodu z Python do PySpark, użycie biblioteki SQL dla Pandas może ułatwić proces przenoszenia.
